### Project 1: Mid-Term 
#### Jessica Bailey | kmr8mp

**Documentation**
<br>
The database created below describes the business transaction processes of a coffee shop chain located in New York City. Data includes things like the date of transaction, which employee was responsible for the transaction, which product was sold, and which store location the transaction occured at, et cetera.
<br>
<br>
***Import necessary libraries***
<br>
The first step is to import all the necessary libraries to run the code. Pandas and SQLAlchemy are necessary for working with the databases and communicating with MySQL Workbench.
<br>
<br>
***Connect to SQL server***
<br>
The code connects to the SQL server using the user ID and password. The source database is set as "employees," a publicly available SQL database that contains employee information such as first and last name, gender, date hired, and birthdate. This database was chosen to be the source because it is modified in the sequential code and used to create the target database, "coffee_sales." "coffee_sales" will contain all relevant information regarding the transactions made at the various coffee shop locations in New York City.
<br>
<br>
***Assign connection variables for MongoDB***
<br>
The code creates the variables needed to connect to the cluster in MongoDB. The database that will be used from the cluster is "sample_analytics," a publicly available MongoDB database that contains the collections "accounts," "customers," and "transactions." For this project, only the "customers" collection is necessary.
<br>
<br>
***Create necessary functions***
<br>
Two functions are created to connect with SQL. The first is called "get_dataframe," and this enables an SQL database to be queried through Jupyter and returned as a Pandas dataframe. The second is called "set_dataframe," which allows the user to push a Pandas dataframe to a SQL database. Three functions are created to connect with MongoDB. The first is called "get_mongo_client," and this allows the user to create a connection with a database in MongoDB. The second is called "get_mongo_dataframe," which connects to the database in MongoDB, extracts a specific collection, converts it to a Pandas dataframe, and closes the connection. The third is called "set_mongo_collections," which connects to the database in MongoDB, reads in data from a JSON file, inserts the data to the specified collection, and closes the connection. All are used in the following code to create the target "coffee_sales" database.
<br>
<br>
***Create the target database***
<br>
The "coffee_sales" database is created by connecting to the SQL server and then executing lines of SQL code inside Jupyter.
<br>
<br>
***Read in data from Excel*** 
<br>
The data pertaining to the transactions and products originated as a .xlsx from Kaggle, a platform for sharing open datasets. The first 500 rows were loaded into Jupyter as a Pandas dataframe, and a column called "transaction_id" was added. The "transaction_id" column helps keep track of all the subsequent columns relating to the business transaction, which is the main focus of the table.
<br>
<br>
***Read in data from MongoDB***
<br>
The data from the "customers" collection in the "sample_analytics" database in MongoDB is read in as a Pandas dataframe.
<br>
<br>
***Create products table***
<br>
A Pandas dataframe containing all relevant information about the products sold at the coffee shops was created from the source data in the excel file. A column called "transaction_id" was added to relate the products to the transactions, and a primary key called "product_key" was added to fully define the table.
<br>
<br>
***Create employees table***
<br>
A Pandas dataframe containing all relevant information about the employees who work at the various locations of the coffee shops was created from the source database "employees" in SQL. "store_id" and "store_location" columns were added to the employees dataframe from the source data in excel to keep track of where each employee works. A "transaction_id" column was added to relate the employees to which transactions they were responsible for, as well as a primary key called "employee_key."
<br>
<br>
***Create customers table***
<br>
A Pandas dataframe containing all relevant information about the customers who purchased products at the various locations of the coffee shops was created from the "customers" collection in MongoDB. A "transaction_id" column was added to relate the customers to which transactions they completed. A "customer_id" column was added to identify each customer, and primary key called "customer_key" was added as well.
<br>
<br>
***Add dataframes to SQL***
<br>
Using the "set_dataframe" function created previously, the products, employees, and customers dataframes were pushed to the empty target database in SQL ("coffee_sales") to be dimension tables.
<br>
<br>
***Execute SQL script from Lab2C to create date dimension table and add to the database in SQL***
<br>
Using the SQL script from Lab2C, the date dimension table is dropped if already present and then created with its corresponding empty columns. Next, a procedure called "PopulateDateDimension" is created to loop through a specified begin and end date, populating the columns in the date dimension table with the necessary data for each day between the two dates. The data is then pushed to the date dimension table in SQL. The procedure is called with the begin date of January 1, 2020, and the end date of January 1, 2024, to encapsulate the dates the transactions were made.
<br>
<br>
***Create fact transactions table***
<br>
The basis for a fact table for the "coffee_sales" database was created by performing an inner join (also called a merge in Pandas) on the products and employees tables, using their respective "transaction_id" columns. Next, an inner join was performed on the fact table and the customer table using the "transaction_id" column. This combines the products, employees, and customers tables in such a way that each row describes one transaction: the transaction ID, the employee responsible for the transaction, the products that were sold, their prices, who purchased the products, and the store location the transaction occured at. Additional columns regarding the date of transaction and the quantity of the transaction were added from the source data in Excel. Unnecessary columns were dropped. These columns were irrelevant to the fact table, as they describe in more detail each of the major components of a transaction (i.e., the employees, products, and customers).
<br>
<br>
***Merge fact transactions table and date info***
<br>
The columns for "date_key" and "full_date" are extracted from the date dimension table, then converted into the proper format. The columns are then renamed in terms of the transaction ("transaction_date_key" and "transaction_date") and are merged to the fact transaction table so that each row describes one transaction: the transaction ID, the employee responsible for the transaction, the products that were sold, their prices, who purchased the products, the store location the transaction occured at, and the date the transaction occurred on. 
<br>
<br>
***Reorder columns and add a primary key***
<br>
The columns in the fact transactions table were reordered to be more logical, with the "transaction_id" column coming first and the foreign keys to the dimension tables coming after. A primary key called "fact_transaction_key" was also added.
<br>
<br>
***Write fact transaction table back to SQL***
<br>
The completed fact transaction table was pushed back to SQL using the "set_dataframe" function.
<br>
<br>
***Validation using SQL queries***
<br>
Three SQL queries were made to validate that the fact transaction table and dimension tables had been successfully pushed back to SQL. The first returns the total number of transactions that an individual employee was responsible for, across all store locations. In this query, the employee ID and name are selected from the employee dimension table, and the transaction quantity from the fact transaction table is summed to calculate the total number of transactions for each employee. The fact transaction table is then joined with the employee dimension table along the employee key, to ensure that each transaction is associated with the correct employee. The results are then grouped by employee and returned in descending order, so the employees who made the most transactions are at the top of the table. The second query returns the most popular product categories for only the store in lower Manhattan. In this query, the product category is selected from the product dimension table, and the sum of the transaction quantity is selected from the fact transaction table. The fact transaction table and the product dimension table are then joined along the product key, ensuring that each transaction is associated with the correct product. The results are filtered to only include transactions that occured in the store located in lower Manhattan, and grouped by product category. The top five most popular products are displayed in descending order. The third query returns the names of the customers who purchased products at the coffee shop located in Astoria, how much money they spent, and on what date the transaction occurred. In this query, the customer name is selected from from the customer dimension table, and the total amount of money they spent is calculated by summing the product of the transaction quanity and the unit price from the fact transaction table. The date is also selected from the date dimension table. The fact transaction table and the customer dimension table are then joined along the customer key, ensuring that each transaction is associated with the correct customer. The fact transaction table is also joined with the date dimension table along the date key, ensuring that each transaction is associated with the correct date. Then the results are filtered to only include transactions that occured in the store located in Astoria, and grouped by customer name as well as date. The results are ordered by descending amount spent, so the customers who spent the most money appear at the top. 

In [1]:
# Import necessary libraries
import os
import numpy
import pandas as pd
import json
import datetime
import certifi
import sqlalchemy
from sqlalchemy import create_engine, text
import pymongo

In [2]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.35
Running PyMongo Version: 4.10.1


In [3]:
# Connect to the SQL server
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "gteruwq4"

# Set the source database and the target database
src_dbname = "employees"
dst_dbname = "coffee_sales"

In [4]:
# Assign connection variables for MongoDB
mongodb_args = {
    "user_name" : "jessicabailey431",
    "password" : "gteruwq4",
    "cluster_name" : "cluster0",
    "cluster_subnet" : "kztaq",
    "cluster_location" : "atlas", 
    "db_name" : "sample_analytics"
}

In [5]:
# Create necessary functions
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()
    
    
def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [6]:
# Create the target database
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

connection.close()

In [7]:
# Read in data from Excel
data_file = r'C:\Users\kmr8mp\Downloads\Academic Weapon\UVA\Classes\Third Year\First Semester\DS Systems\Coffee Shop Sales.xlsx'
df_coffee_sales_src = pd.read_excel(data_file, nrows=500, header=0, index_col=0)
df_coffee_sales_src.index.name = None
df_coffee_sales_src.insert(0, "transaction_id", range(1, df_coffee_sales_src.shape[0]+1))
df_coffee_sales_src.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
1,1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg
2,2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
3,3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
4,4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm
5,5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg


In [8]:
# Read in data from MongoDB
client = get_mongo_client(**mongodb_args)

query = {} 
collection = "customers"

df_customers = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_customers.head()

,username,name,address,birthdate,email,accounts,tier_and_details,active
0,petergilbert,Angela Campbell,"4068 Espinoza Mills\nWest Jessica, WV 60790",1991-12-15 18:03:16,jocelyn67@yahoo.com,"[260499, 946116, 588389, 293111, 126444, 678107]",{'1768a44b906c4303b765ea38b1b4bbb2': {'tier': ...,NaN
1,rodriguezjeffrey,Steven Campbell,"69809 Morris Extensions\nTaylorhaven, ME 37916",1976-10-31 19:39:51,kristen30@yahoo.com,"[607567, 429282]",{},NaN
2,valenciajennifer,Lindsay Cowan,Unit 1047 Box 4089\nDPO AA 57348,1994-02-19 23:46:27,cooperalexis@hotmail.com,[116508],{'c06d340a4bad42c59e3b6665571d2907': {'tier': ...,NaN
3,wesley20,James Sanchez,"8681 Karen Roads Apt. 096\nLowehaven, IA 19798",1973-01-13 16:17:26,josephmacias@hotmail.com,[987709],{},NaN
4,hmyers,Dana Clarke,"50047 Smith Point Suite 162\nWilkinsstad, PA 0...",1969-06-21 02:39:20,vcarter@hotmail.com,"[627629, 55958, 771641]",{'4c207e65857742f89d8155139b24c0f0': {'tier': ...,NaN


In [9]:
# Create products table
df_products = df_coffee_sales_src[['product_id','unit_price','product_category','product_type','product_detail']]
df_products.index.name = None
df_products.insert(0, "transaction_id", range(1, df_products.shape[0]+1))
df_products.insert(0, "product_key", range(1, df_products.shape[0]+1))
df_products.head()

,product_key,transaction_id,product_id,unit_price,product_category,product_type,product_detail
1,1,1,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg
2,2,2,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
3,3,3,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
4,4,4,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm
5,5,5,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg


In [10]:
# Create employees table
sql_employees = "SELECT * FROM employees LIMIT 501;"
df_employees = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_employees))
df_employees = df_employees.drop(0)
df_employees.index.name = None
drop_cols = ['birth_date','gender']
df_employees.drop(drop_cols, axis=1, inplace=True)
df_employees[['store_id','store_location']] = df_coffee_sales_src[['store_id','store_location']]
df_employees.rename(columns={"emp_no":"employee_id"}, inplace=True)
df_employees.rename(columns={"first_name":"employee_first_name"}, inplace=True)
df_employees.rename(columns={"last_name":"employee_last_name"}, inplace=True)
df_employees.insert(0, "transaction_id", range(1, df_employees.shape[0]+1))
df_employees.insert(0, "employee_key", range(1, df_employees.shape[0]+1))
df_employees.head()

,employee_key,transaction_id,employee_id,employee_first_name,employee_last_name,hire_date,store_id,store_location
1,1,1,10002,Bezalel,Simmel,1985-11-21,5,Lower Manhattan
2,2,2,10003,Parto,Bamford,1986-08-28,5,Lower Manhattan
3,3,3,10004,Chirstian,Koblick,1986-12-01,5,Lower Manhattan
4,4,4,10005,Kyoichi,Maliniak,1989-09-12,5,Lower Manhattan
5,5,5,10006,Anneke,Preusig,1989-06-02,5,Lower Manhattan


In [11]:
# Create customers table
df_customers = df_customers.drop(0)
drop_cols = ['username','birthdate','active','accounts','tier_and_details']
df_customers.drop(drop_cols, axis=1, inplace=True)
df_customers.rename(columns={"name":"customer_name"}, inplace=True)
df_customers.insert(0, "customer_key", range(1, df_customers.shape[0]+1))
df_customers.insert(0, "transaction_id", range(1, df_customers.shape[0]+1))
df_customers.insert(0, "customer_id", range(1, df_customers.shape[0]+1))
df_customers.head()

,customer_id,transaction_id,customer_key,customer_name,address,email
1,1,1,1,Steven Campbell,"69809 Morris Extensions\nTaylorhaven, ME 37916",kristen30@yahoo.com
2,2,2,2,Lindsay Cowan,Unit 1047 Box 4089\nDPO AA 57348,cooperalexis@hotmail.com
3,3,3,3,James Sanchez,"8681 Karen Roads Apt. 096\nLowehaven, IA 19798",josephmacias@hotmail.com
4,4,4,4,Dana Clarke,"50047 Smith Point Suite 162\nWilkinsstad, PA 0...",vcarter@hotmail.com
5,5,5,5,Ashley Lopez,"18637 Jessica Ridge Apt. 157\nGrossberg, ME 84127",michael16@hotmail.com


In [12]:
# Add dataframes to sql
db_operation = "insert"

tables = [('dim_products', df_products, 'product_key'),
          ('dim_employees', df_employees, 'employee_key'),
          ('dim_customers', df_customers, 'customer_key')]

for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [ ]:
# Execute sql script from Lab2c to create date dimension table and add to the database in sql

In [13]:
# Create fact transactions table
df_fact_transactions = pd.merge(df_employees,df_products,on='transaction_id',how='inner')
df_fact_transactions = pd.merge(df_fact_transactions,df_customers,on='transaction_id',how='inner')
df_transactions = df_coffee_sales_src[['transaction_id','transaction_date','transaction_qty']]
df_fact_transactions = pd.merge(df_fact_transactions,df_transactions,on='transaction_id',how='inner')

# Drop unnecessary columns
drop_cols = ['employee_id','employee_first_name','employee_last_name','hire_date','product_id','product_category','product_type','product_detail','customer_id','customer_name','address','email']
df_fact_transactions.drop(drop_cols, axis=1, inplace=True)
df_fact_transactions.head()

,employee_key,transaction_id,store_id,store_location,product_key,unit_price,customer_key,transaction_date,transaction_qty
0,1,1,5,Lower Manhattan,1,3.0,1,2023-01-01,2
1,2,2,5,Lower Manhattan,2,3.1,2,2023-01-01,2
2,3,3,5,Lower Manhattan,3,4.5,3,2023-01-01,2
3,4,4,5,Lower Manhattan,4,2.0,4,2023-01-01,1
4,5,5,5,Lower Manhattan,5,3.1,5,2023-01-01,2


In [14]:
# Merge fact transaction table and date info
sql_dim_date = "SELECT date_key, full_date FROM coffee_sales.dim_date;"
df_dim_date = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_dim_date))
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date

df_dim_transaction_date = df_dim_date.rename(columns={"date_key" : "transaction_date_key", "full_date" : "transaction_date"})
df_fact_transactions.transaction_date = df_fact_transactions.transaction_date.astype('datetime64[ns]').dt.date

df_fact_transactions = pd.merge(df_fact_transactions, df_dim_transaction_date, on='transaction_date', how='left')
df_fact_transactions.drop(['transaction_date'], axis=1, inplace=True)
df_fact_transactions.head()

,employee_key,transaction_id,store_id,store_location,product_key,unit_price,customer_key,transaction_qty,transaction_date_key
0,1,1,5,Lower Manhattan,1,3.0,1,2,20230101
1,2,2,5,Lower Manhattan,2,3.1,2,2,20230101
2,3,3,5,Lower Manhattan,3,4.5,3,2,20230101
3,4,4,5,Lower Manhattan,4,2.0,4,1,20230101
4,5,5,5,Lower Manhattan,5,3.1,5,2,20230101


In [15]:
# Reorder columns and add a primary key
ordered_cols = ['transaction_id','employee_key','product_key','customer_key','transaction_date_key','transaction_qty','unit_price','store_id','store_location']
df_fact_transactions = df_fact_transactions[ordered_cols]
df_fact_transactions.insert(0,"fact_transaction_key",range(1,df_fact_transactions.shape[0]+1))
df_fact_transactions.head()

,fact_transaction_key,transaction_id,employee_key,product_key,customer_key,transaction_date_key,transaction_qty,unit_price,store_id,store_location
0,1,1,1,1,1,20230101,2,3.0,5,Lower Manhattan
1,2,2,2,2,2,20230101,2,3.1,5,Lower Manhattan
2,3,3,3,3,3,20230101,2,4.5,5,Lower Manhattan
3,4,4,4,4,4,20230101,1,2.0,5,Lower Manhattan
4,5,5,5,5,5,20230101,2,3.1,5,Lower Manhattan


In [16]:
# Write fact transaction table back to sql
table_name = "fact_transactions"
primary_key = "fact_transaction_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_fact_transactions, table_name, primary_key, db_operation)
df_fact_transactions.head()

,fact_transaction_key,transaction_id,employee_key,product_key,customer_key,transaction_date_key,transaction_qty,unit_price,store_id,store_location
0,1,1,1,1,1,20230101,2,3.0,5,Lower Manhattan
1,2,2,2,2,2,20230101,2,3.1,5,Lower Manhattan
2,3,3,3,3,3,20230101,2,4.5,5,Lower Manhattan
3,4,4,4,4,4,20230101,1,2.0,5,Lower Manhattan
4,5,5,5,5,5,20230101,2,3.1,5,Lower Manhattan


In [17]:
# Author a query that returns the total number of transactions that each employee is responsible for
sql_test1 = """
SELECT 
    e.employee_key,               
    e.employee_first_name,                
    e.employee_last_name,                   
    SUM(ft.transaction_qty) AS total_transactions  
FROM 
    coffee_sales.fact_transactions AS ft  
JOIN 
    coffee_sales.dim_employees AS e      
ON 
    ft.employee_key = e.employee_key 
GROUP BY 
    e.employee_key,                  
    e.employee_first_name, 
    e.employee_last_name
ORDER BY 
    total_transactions DESC;          
""".format(dst_dbname)

df_test1 = get_dataframe(user_id, pwd, host_name, dst_dbname, text(sql_test1))
df_test1.head()

,employee_key,employee_first_name,employee_last_name,total_transactions
0,1,Bezalel,Simmel,2.0
1,2,Parto,Bamford,2.0
2,3,Chirstian,Koblick,2.0
3,5,Anneke,Preusig,2.0
4,8,Sumant,Peac,2.0


In [18]:
# Author a query that returns the most popular product category sold at the coffee shop in Lower Manhattan (store_id = 5)
sql_test2 = """
SELECT 
p.product_category,                       
SUM(ft.transaction_qty) AS total_sold     
FROM 
    coffee_sales.fact_transactions AS ft
JOIN 
    coffee_sales.dim_products AS p                            
ON 
    ft.product_key = p.product_key            
WHERE 
    ft.store_id = 5                          
GROUP BY 
    p.product_category                        
ORDER BY 
    total_sold DESC                           
LIMIT 5;                                          
""".format(dst_dbname)

df_test2 = get_dataframe(user_id, pwd, host_name, dst_dbname, text(sql_test2))
df_test2.head()

,product_category,total_sold
0,Coffee,117.0
1,Tea,85.0
2,Bakery,33.0
3,Drinking Chocolate,20.0


In [19]:
# Author a query that returns the names of the customers, how much money they spent at the coffee shop located in Astoria (store id = 3), and on what date the transaction occurred
sql_test3 = """
SELECT 
    c.customer_name,                                  
    SUM(ft.transaction_qty * ft.unit_price) AS total_spent,
    d.full_date AS transaction_date  
FROM 
    coffee_sales.fact_transactions AS ft
JOIN 
    coffee_sales.dim_customers AS c                                   
ON 
    ft.customer_key = c.customer_key
JOIN 
    coffee_sales.dim_date AS d
ON 
    ft.transaction_date_key = d.date_key                   
WHERE 
    ft.store_id = 3                                  
GROUP BY 
    c.customer_name, 
    d.full_date                                  
ORDER BY 
    total_spent DESC;                                                     
""".format(dst_dbname)


df_test3 = get_dataframe(user_id, pwd, host_name, dst_dbname, text(sql_test3))
df_test3.head()

,customer_name,total_spent,transaction_date
0,Michael Harris,9.5,2023-01-01
1,Michelle Phillips,8.5,2023-01-01
2,Jacob Green,8.5,2023-01-01
3,Justin Thompson,8.5,2023-01-01
4,Lauren Hernandez,8.5,2023-01-01
